In [270]:
#UNDER CONSTRUCTION

import os
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import pandas_datareader
import matplotlib.pyplot as plt
import datetime
import tweepy
import twitterAPI
import sklearn
import pprint
import requests
from bs4 import BeautifulSoup
import json
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as vaderSA
import tqdm

pp = pprint.PrettyPrinter(indent=4)

path = "/Users/Matzekoek/Documents/Webhose.io - World News Articles sep-oct 2015/Webhose.io - World News Articles Sep 2015(2)"
os.chdir(path)

alphaVantageKey = '6AEOHIB29K740TG5'
twitterapikey = 'jNjZ10xjmy3f3pMTojP9kjNWY'
twitterapisecret = 'szou2LYaqQPXE9XMEKYnA3ETz2NZoHZcHuylswb7mgJxXCWPIE'
twitteraccesstoken = '1126856371989884929-m4K4b5Bm40yzIkHTr714a2wMRF5WaZ'
twitteraccesstokensecret = '49P9ZAs2w42IqJ059Fl5ylVvinnuF4wIQPP12xnDxxSHW'

ModuleNotFoundError: No module named 'twitterAPI'

In [268]:
pip install twitterapi

  Stored in directory: /Users/Matzekoek/Library/Caches/pip/wheels/1f/f9/82/8b46b92e9860fccb75ca97af2fa0bd80a88e02a2001a975537
Successfully built twitterapi
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Program design:


## NEWS

#access Twitter and download all FB tweets
#store news with headline, content, date, category(, no of newssources?, virality?)
#clean news data (universal time, correct for stock market closing, correct for weekends, remove unwanted characters from text)
#analyze sentiment of news content

## STOCK DATA

#access EOD stock data for FB stock from NYSE
#for specific time/date, predict if up/down on basis of sum of valence for that day
#(or weighted by # of sources/category/virality etc)

## IDEAS
#integrate NY weather data
#query twitter for keywords from headlines, and analyze sentiment
#widely covered, very viral, very negative news data can have negative effect on stock market (terror attacks, etc)


In [ ]:
#TO DO:

#access intraday stock data for NYSE
#for specific time/date, predict if up/down on basis of sum of valence for that hour/10min 
#(or weighted by # of sources/category/virality etc)

#DONE:

#Get news data
#store news with headline, content, date, category, no of newssources?, virality?
#clean news data (universal time, correct for news after trading closes, remove unwanted characters from text)
#analyze sentiment of content
#extract timezone, check for abberations



In [267]:
# twitterapikey = 
# twitterapisecret = 
# twitteraccesstoken = 
# twitteraccesstokensecret =  

CONSUMER_KEY = 'jNjZ10xjmy3f3pMTojP9kjNWY'
CONSUMER_SECRET = 'szou2LYaqQPXE9XMEKYnA3ETz2NZoHZcHuylswb7mgJxXCWPIE'
ACCESS_KEY = '1126856371989884929-m4K4b5Bm40yzIkHTr714a2wMRF5WaZ'
ACCESS_SECRET = '49P9ZAs2w42IqJ059Fl5ylVvinnuF4wIQPP12xnDxxSHW'

auth = tweepy.auth.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)
search_results = api.search(q="facebook", count=100)

pp.pprint(search_results['statuses'])

TypeError: list indices must be integers or slices, not str

In [265]:
dflist = []

for i in search_results:
    dflist.append(i)

pp.pprint(search_results.keys())

AttributeError: 'SearchResults' object has no attribute 'keys'

In [168]:
newsdf = pd.read_csv('Webhose.io - World News Articles Sep 2015.csv')

#newsdf.to_csv('Webhose.io - World News Articles Sep 2015.csv')

In [169]:
newsdf.head()

,Unnamed: 0,index,date,title,source,source_section,language,country,text
0,0,0,2015-09-08T13:20:00.000+03:00,"Germany to receive more than 800,000 refugees ...",channelnewsasia.com,Channel NewsAsia World News,english,SG,"A \nBERLIN: More than 800,000 refugees will co..."
1,1,0,2015-09-04T10:39:00.000+03:00,US promising help to keep Iran in check in Mid...,fox30jax.com,World News,english,US,"Updated: 3:39 a.m. Friday, Sept. 4, 2015 | Pos..."
2,2,0,2015-09-20T03:00:00.000+03:00,Carly Fiorina is a rock-star Republican but Ra...,theguardian.com,World news | The Guardian,english,US,No matter where Carly Fiorina went on Mackinac...
3,3,0,2015-09-30T03:00:00.000+03:00,Singapore 'respects Indonesia's sovereignty',wn.com,WN.com - World News,english,GB,Singapore respects Indonesia 's sovereignty wi...
4,4,0,2015-09-25T19:04:00.000+03:00,Activist Pressure Makes Lonestar Hike Price fo...,wsj.com,"The Wall Street Journal &amp; Breaking News, B...",english,US,Private-equity firm Lone Star Funds has bowed ...


In [107]:
#prepare files for processing

files = []
for filename in os.listdir(path):
    files.append(str(filename))

In [108]:
#Get relevant news features

def getNewsFeatures(functionfile):
    
    jsonfile = json.load(open(str(functionfile)))
    
    date =           jsonfile['published']
    title =          jsonfile['title']
    source =         jsonfile['thread']['site']
    source_section = jsonfile['thread']['section_title']
    language =       jsonfile['language']
    country =        jsonfile['thread']['country']
    text =           jsonfile['text']
    return [date, title, source, source_section, language, country, text]
    #pp.print (date, title, source, source_section, language, country, text)

#samplefile = 'news_0000001.json'
#samplejson = json.load(open('news_0000001.json'))
#pp.pprint(getNewsFeatures(samplefile))

In [109]:
#Convert raw data to dataframe

columns = ['date', 'title', 'source','source_section','language','country','text']

newsdf = pd.DataFrame(columns = columns)

for file in files:
    
    jsondf = pd.DataFrame(getNewsFeatures(file)).T
    jsondf.columns = columns
    newsdf = newsdf.append(jsondf)
    print(file)

newsdf.reset_index(inplace = True) 

news_0014010.json
news_0023363.json
news_0008033.json
news_0000403.json
news_0004913.json
news_0019622.json
news_0018930.json
news_0029294.json
news_0005601.json
news_0023699.json
news_0026161.json
news_0011212.json
news_0008463.json
news_0023733.json
news_0014440.json
news_0013385.json
news_0022821.json
news_0019788.json
news_0006884.json
news_0007796.json
news_0000053.json
news_0002594.json
news_0005251.json
news_0019272.json
news_0011642.json
news_0016187.json
news_0008199.json
news_0026531.json
news_0010950.json
news_0008930.json
news_0015601.json
news_0014913.json
news_0009622.json
news_0022572.json
news_0001212.json
news_0018033.json
news_0004010.json
news_0028485.json
news_0022088.json
news_0027770.json
news_0026862.json
news_0010403.json
news_0022122.json
news_0009272.json
news_0012594.json
news_0015251.json
news_0018199.json
news_0000950.json
news_0001642.json
news_0006187.json
news_0004440.json
news_0003385.json
news_0018463.json
news_0017796.json
news_0010053.json
news_00097

news_0014694.json
news_0030631.json
news_0000287.json
news_0007542.json
news_0005085.json
news_0002740.json
news_0029410.json
news_0003852.json
news_0016353.json
news_0011496.json
news_0021020.json
news_0013501.json
news_0025960.json
news_0024672.json
news_0007112.json
news_0030261.json
news_0029040.json
news_0002310.json
news_0021470.json
news_0024388.json
news_0017811.json
news_0016703.json
news_0019399.json
news_0000442.json
news_0007387.json
news_0008072.json
news_0012886.json
news_0023322.json
news_0013794.json
news_0014051.json
news_0016596.json
news_0011253.json
news_0008588.json
news_0026120.json
news_0005640.json
news_0002185.json
news_0018971.json
news_0019663.json
news_0004952.json
news_0000012.json
news_0022860.json
news_0014401.json
news_0023772.json
news_0008422.json
news_0010911.json
news_0023288.json
news_0026570.json
news_0011603.json
news_0019233.json
news_0028997.json
news_0005210.json
news_0029685.json
news_0018588.json
news_0006596.json
news_0001253.json
news_00225

news_0007503.json
news_0030670.json
news_0013110.json
news_0024263.json
news_0021061.json
news_0024799.json
news_0016312.json
news_0003813.json
news_0029451.json
news_0002701.json
news_0001984.json
news_0030220.json
news_0007153.json
news_0000696.json
news_0024633.json
news_0025921.json
news_0014285.json
news_0013540.json
news_0011087.json
news_0016742.json
news_0017850.json
news_0021431.json
news_0002351.json
news_0029001.json
news_0005494.json
news_0020934.json
news_0011290.json
news_0014568.json
news_0016555.json
news_0022909.json
news_0021626.json
news_0002146.json
news_0029216.json
news_0005683.json
news_0004991.json
news_0030037.json
news_0007344.json
news_0000481.json
news_0005379.json
news_0012845.json
news_0026419.json
news_0024424.json
news_0010878.json
news_0014092.json
news_0013757.json
news_0021276.json
news_0014138.json
news_0016105.json
news_0028954.json
news_0029646.json
news_0002516.json
news_0007714.json
news_0018818.json
news_0005729.json
news_0030467.json
news_00068

news_0011947.json
news_0003583.json
news_0004246.json
news_0031108.json
news_0018265.json
news_0007893.json
news_0028329.json
news_0003079.json
news_0006781.json
news_0001044.json
news_0022724.json
news_0009474.json
news_0020719.json
news_0015457.json
news_0023836.json
news_0012392.json
news_0026767.json
news_0013429.json
news_0027875.json
news_0025848.json
news_0011414.json
news_0019024.json
news_0030349.json
news_0005007.json
news_0029492.json
news_0000205.json
news_0029168.json
news_0002238.json
news_0009927.json
news_0014616.json
news_0021558.json
news_0015904.json
news_0017939.json
news_0023565.json
news_0008635.json
news_0013079.json
news_0016781.json
news_0011044.json
news_0017893.json
news_0026337.json
news_0005457.json
news_0002392.json
news_0019474.json
news_0030719.json
news_0001947.json
news_0000655.json
news_0002668.json
news_0029538.json
news_0007190.json
news_0021108.json
news_0008265.json
news_0023135.json
news_0013583.json
news_0014246.json
news_0029257.json
news_00021

news_0029980.json
news_0018224.json
news_0031149.json
news_0004207.json
news_0028692.json
news_0011906.json
news_0027567.json
news_0012629.json
news_0010614.json
news_0023877.json
news_0015416.json
news_0020758.json
news_0009435.json
news_0022765.json
news_0001005.json
news_0003038.json
news_0028368.json
news_0002783.json
news_0005046.json
news_0030308.json
news_0019065.json
news_0003891.json
news_0025809.json
news_0011455.json
news_0013468.json
news_0016390.json
news_0027834.json
news_0026726.json
news_0017978.json
news_0008674.json
news_0023524.json
news_0021519.json
news_0015945.json
news_0014657.json
news_0013192.json
news_0009966.json
news_0002279.json
news_0029129.json
news_0007581.json
news_0000244.json
news_0030758.json
news_0019435.json
news_0029083.json
news_0005416.json
news_0026376.json
news_0011005.json
news_0013038.json
news_0014207.json
news_0023174.json
news_0008224.json
news_0021149.json
news_0029579.json
news_0002629.json
news_0000614.json
news_0001906.json
news_00115

...

news_0022967.json
news_0014506.json
news_0021648.json
news_0008525.json
news_0023675.json
news_0000115.json
news_0002128.json
news_0029278.json
news_0003693.json
news_0004156.json
news_0031218.json
news_0018175.json
news_0002981.json
news_0010545.json
news_0024919.json
news_0017280.json
news_0026924.json
news_0012578.json
news_0027636.json
news_0009764.json
news_0022434.json
news_0016868.json
news_0014855.json
news_0020409.json
news_0015747.json
news_0012082.json
news_0008876.json
news_0006491.json
news_0003369.json
news_0028039.json
news_0001354.json
news_0031648.json
news_0018525.json
news_0028193.json
news_0004506.json
news_0027266.json
news_0010115.json
news_0012128.json
news_0015317.json
news_0022064.json
news_0009334.json
news_0020059.json
news_0028469.json
news_0003739.json
news_0001704.json
news_0000816.json
news_0014910.json
news_0022571.json
news_0009621.json
news_0027289.json
news_0008933.json
news_0015602.json
news_0001211.json
news_0004013.json
news_0028486.json
news_00180

news_0026066.json
news_0011315.json
news_0013328.json
news_0004814.json
news_0030448.json
news_0019725.json
news_0006829.json
news_0029393.json
news_0018837.json
news_0005706.json
news_0002539.json
news_0029669.json
news_0000504.json
news_0014117.json
news_0008134.json
news_0023264.json
news_0021259.json
news_0011745.json
news_0016080.json
news_0013778.json
news_0026436.json
news_0010857.json
news_0002493.json
news_0005356.json
news_0030018.json
news_0019375.json
news_0006983.json
news_0007691.json
news_0029239.json
news_0002169.json
news_0000154.json
news_0023634.json
news_0008564.json
news_0021609.json
news_0014547.json
news_0022926.json
news_0013282.json
news_0027677.json
news_0026965.json
news_0012539.json
news_0010504.json
news_0024958.json
news_0018134.json
news_0031259.json
news_0004117.json
news_0028582.json
news_0001315.json
news_0028078.json
news_0003328.json
news_0008837.json
news_0015706.json
news_0014814.json
news_0020448.json
news_0022475.json
news_0009725.json
news_00168

news_0017015.json
news_0029844.json
news_0028756.json
news_0003406.json
news_0019908.json
news_0006604.json
news_0004639.json
news_0031577.json
news_0007916.json
news_0012217.json
news_0027159.json
news_0025164.json
news_0016254.json
news_0014269.json
news_0011591.json
news_0021127.json
news_0005182.json
news_0029517.json
news_0002647.json
news_0003955.json
news_0001968.json
news_0030736.json
news_0005478.json
news_0000380.json
news_0007445.json
news_0015881.json
news_0024325.json
news_0026318.json
news_0013056.json
news_0014793.json
news_0021577.json
news_0017916.json
news_0009908.json
news_0016604.json
news_0014639.json
news_0002217.json
news_0029147.json
news_0005028.json
news_0007015.json
news_0030366.json
news_0013406.json
news_0025867.json
news_0024775.json
news_0026748.json
news_0007202.json
news_0030171.json
news_0013611.json
news_0024562.json
news_0012903.json
news_0024098.json
news_0021760.json
news_0016413.json
news_0020872.json
news_0002000.json
news_0029350.json
news_00069

news_0022377.json
news_0009027.json
news_0015004.json
news_0017039.json
news_0027525.json
news_0013979.json
news_0011944.json
news_0025518.json
news_0010656.json
news_0017193.json
news_0018266.json
news_0003580.json
news_0006278.json
news_0004245.json
news_0006782.json
news_0001047.json
news_0007890.json
news_0015454.json
news_0021808.json
news_0023835.json
news_0012391.json
news_0017469.json
news_0009477.json
news_0022727.json
news_0018289.json
news_0001552.json
news_0006297.json
news_0009162.json
news_0013996.json
news_0022232.json
news_0012684.json
news_0015141.json
news_0017486.json
news_0010343.json
news_0027030.json
news_0009498.json
news_0004750.json
news_0003095.json
news_0019861.json
news_0018773.json
news_0005842.json
news_0001102.json
news_0023970.json
news_0015511.json
news_0022662.json
news_0009532.json
news_0011801.json
news_0027460.json
news_0022398.json
news_0010713.json
news_0018323.json
news_0029887.json
news_0004300.json
news_0028795.json
news_0019498.json
news_00074

news_0002780.json
news_0007078.json
news_0005045.json
news_0026725.json
news_0024718.json
news_0011456.json
news_0016393.json
news_0027837.json
news_0014654.json
news_0009965.json
news_0013191.json
news_0016669.json
news_0023527.json
news_0008677.json
news_0015946.json
news_0007582.json
news_0000247.json
news_0029080.json
news_0007428.json
news_0005415.json
news_0019436.json
news_0011006.json
news_0026375.json
news_0024348.json
news_0008227.json
news_0023177.json
news_0014204.json
news_0016239.json
news_0001905.json
news_0003938.json
news_0000617.json
news_0018677.json
news_0005946.json
news_0004654.json
news_0003191.json
news_0019965.json
news_0006669.json
news_0025109.json
news_0027134.json
news_0017582.json
news_0010247.json
news_0012780.json
news_0017078.json
news_0015045.json
news_0009066.json
news_0022336.json
news_0013892.json
news_0001456.json
news_0006393.json
news_0029829.json
news_0004204.json
news_0028691.json
news_0006239.json
news_0029983.json
news_0018227.json
news_00106

In [173]:
twodaydelta = datetime.timedelta(days=2)
newsdf['date_weekday'] = newsdf.datetime_utc_corrected.apply(lambda dt: dt.weekday())

mask = newsdf.date_weekday == 5
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + twodaydelta)
                                 
mask = newsdf.date_weekday == 6
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + daydelta)

In [170]:
#correct news timestamp for: timezone, published after closed trading, weekend

newsdf['datetime'] = pd.to_datetime(newsdf['date'].str[0:19], format='%Y-%m-%dT%H:%M:%S')

utcdiff = datetime.timedelta(hours=-7)
newsdf['datetime_utc_corrected'] = newsdf['datetime'] + utcdiff
newsdf['date_1700_utc'] = pd.to_datetime(newsdf['datetime_utc_corrected'].astype(str).str[0:10] + ' ' + '17:00:00', format='%Y-%m-%d %H:%M:%S')

daydelta = datetime.timedelta(days = 1)
mask = newsdf.datetime_utc_corrected > newsdf.date_1700_utc
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'].apply(lambda dt: dt.replace(hour=9, minute=0)) + daydelta)

twodaydelta = datetime.timedelta(days=2)
newsdf['date_weekday'] = newsdf.datetime_utc_corrected.apply(lambda dt: dt.weekday())

mask = newsdf.date_weekday == 5
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + twodaydelta)

mask = newsdf.date_weekday == 6
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + daydelta)


newsdf['text'] = newsdf['text'].str.replace(r'\n', ' ')
newsdf['text'] = newsdf['text'].str.replace("'", ' ')
newsdf['text'] = newsdf['text'].str.replace('"', ' ')
newsdf['text'] = newsdf['text'].str.replace('  ', ' ')

In [397]:
#Extract compound sentiment score with VADER, create sentiment dataframe, add to newsdf

analyser = vaderSA()

textlist = newsdf['text'].tolist()
sentimentdf = pd.DataFrame(columns = ['text_compound_sentiment_score'])

sentimentlist = []
for text in textlist:
    sentimentlist.append(analyser.polarity_scores(str(text))['compound'])
    
sentimentdf = pd.DataFrame(sentimentlist, columns = ['text_compound_sentiment'])

newsdf['text_compound_sentiment'] = sentimentdf['text_compound_sentiment']

In [235]:
# Get Finance Data from Alpha Vantage

#ts = TimeSeries(key='alphaVantageKey', output_format='pandas')
#stockdf, meta_data = ts.get_daily_adjusted(symbol='FB', outputsize='full')

#stockdf[(stockdf['date']>startdate) & (stockdf['date']<enddate)]

# print (data)
# data['5. adjusted close'].plot()
# plt.title('Stock chart')
# plt.show()

startdate = '2015-09-01'
enddate = '2015-09-30'

stockdf = stockdf[startdate:enddate]

stockdf['up'] = stockdf['5. adjusted close'] > stockdf['1. open']
stockdf['down'] = stockdf['5. adjusted close'] < stockdf['1. open']



# print (stockdf)
# stockdf['5. adjusted close'].plot()
# plt.title('Stock chart')
# plt.show()



In [236]:
stockdf.head(15)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,up,down
date,,,,,,,,,,
2015-09-01,86.99,89.39,86.50,87.23,87.23,36097370.0,0.0,1.0,True,False
2015-09-02,88.82,89.89,87.63,89.89,89.89,27190527.0,0.0,1.0,True,False
2015-09-03,89.76,89.82,87.73,88.15,88.15,27048198.0,0.0,1.0,False,True
2015-09-04,87.20,88.39,86.70,88.26,88.26,27328667.0,0.0,1.0,True,False
2015-09-08,89.69,90.24,88.83,89.53,89.53,27012543.0,0.0,1.0,False,True
2015-09-09,90.78,91.98,90.17,90.44,90.44,32551123.0,0.0,1.0,False,True
2015-09-10,90.07,92.06,89.63,91.98,91.98,26437398.0,0.0,1.0,True,False
2015-09-11,91.68,92.08,91.13,92.05,92.05,21185049.0,0.0,1.0,True,False
2015-09-14,92.31,92.62,91.60,92.31,92.31,20127871.0,0.0,1.0,False,False


In [ ]:
'''

getNewsFeatures(samplefile)


path = "/Users/Matzekoek/Documents/Webhose.io - World News Articles sep-oct 2015/Webhose.io - World News Articles Sep 2015"
os.chdir(path)

columns = ['date', 'title', 'source','source_section','language','country','text', 'sentiment']

newsdf = pd.DataFrame(columns = columns)


textlist = newsdf['text'].tolist()
sentimentdf = pd.DataFrame(columns = ['text_compound_sentiment_score'])

sentimentlist = []
for text in textlist:
    sentimentlist.append(analyser.polarity_scores(str(text))['compound'])

os.getcwd()

pp.pprint(samplejson['published'])
pp.pprint(samplejson['title'])
pp.pprint(samplejson['thread']['site'])
pp.pprint(samplejson['thread']['section_title'])
pp.pprint(samplejson['language'])
pp.pprint(samplejson['thread']['country'])
pp.pprint(samplejson['text'])


#s = pd.Series(getNewsFeatures(file), index=df.columns)
    
testdf = pd.DataFrame(getNewsFeatures(samplefile)).T
testdf.columns = columns

samplefile2 = 'news_0000002.json'

testdf2 = pd.DataFrame(getNewsFeatures(samplefile2)).T
testdf2.columns = columns

testdf = testdf.append(testdf2)
testdf.head()  


    
    
# year = str(newsdf.iloc[0:1]['date'])[5:9]
# month = str(newsdf.iloc[0:1]['date'])[11:12]
# day = str(newsdf.iloc[0:1]['date'])[13:15]
# hour = str(newsdf.iloc[0:1]['date'])[16:18]
# minute = str(newsdf.iloc[0:1]['date'])[19:21]
# secs = str(newsdf.iloc[0:1]['date'])[22:24]

#date = str(newsdf.iloc[0:1]['date'])[5:15]
#time = str(newsdf.iloc[0:1]['date'])[16:24]

#newsdf['singledate'] = str(newsdf.iloc[0:1]['date'])[5:15]
# FOUT   newsdf['time'] = (str(newsdf['date'])[16:24])

print(newsdf['datetime'].iloc[0:1].item())
utcdif = datetime.timedelta(hours=-5)

print(newsdf['datetime'].iloc[0:1].item() + utcdif)


#print(file)
#soup = BeautifulSoup(open(str(os.path.join(path,file))))

#newsdf['date_extract'] = newsdf['date'].str[0:10]
#newsdf['time_extract'] = newsdf['date'].str[11:19]
#newsdf['datetime_string'] = newsdf['date_extract'] + ' ' + newsdf['time_extract']


newsdf[newsdf['datetime_utc_corrected'] > newsdf['date_1700_utc']]['datetime_utc_corrected'] = 

# twodaydelta = datetime.timedelta(days=2)
# newsdf['date_weekday'] = newsdf.datetime_utc_corrected.weekday()

#newsdf['datetime_1700'] = pd.to_datetime((newsdf['date_extract'] + ' ' + '17:00:00'), format='%Y-%m-%d %H:%M:%S')

#newsdf['utc_difference_extract'] = newsdf['date'].str[23:29]

#newsdf['text_compound_sentiment'] = analyser.polarity_scores(str(newsdf['text']))['compound']
# werkt niet, geeft waarde voor hele textbody van alles samen, zelfde als analyser.polarity_scores(str(newsdf['text'].iloc[:]))
#newsdf['text_compound_sentiment'] = newsdf['text'].apply(analyser.polarity_scores())
# for text in newsdf['text']:
#     newsdf['text_compound_sentiment'] = analyser.polarity_scores(text)

#checked with newsdf['utc_difference_extract'].value_counts(), all from same timestamp

#newsdf.to_csv('Webhose.io - World News Articles Sep 2015.csv')

#fbstockdata = quandl.get("EOD/FB", start_date=startdate, end_date=enddate)

'''